In [418]:
import numpy as np # 자연로그와 pi연산을 위한 라이브러리
from PIL import Image # 이미지 파일을 가져오기 위한 라이브러리

## Log

gaussian kernel 생성 함수

In [419]:
def gaussian_kernel(sigma, size): # size*size 가우시안 커널 생성
    kernel = [[0]*size for _ in range(size)]
    center = (size - 1) / 2

    for x in range(size):
        for y in range(size):
            exponent = -((x - center)**2 + (y - center)**2) / (2 * sigma**2)
            kernel[x][y] = (1 / (2 * np.pi * sigma**2)) * np.exp(exponent)

    return kernel

laplacian kernel 생성 함수

In [420]:
def laplacian_kernel():
    kernel = [[0, 1, 0], [1, -4, 1], [0, 1, 0]]
    return kernel

padding

In [421]:
def zero_padding(matrix, pad_size, m, n):
    padded_matrix = [[0]*(n + 2*pad_size) for _ in range(m + 2*pad_size)] # 패딩 크기만큼 행과 열을 각각 양쪽으로 늘림

    # 원본 행렬을 새로운 행렬의 중앙에 배치
    for i in range(m):
        for j in range(n):
            padded_matrix[i + pad_size][j + pad_size] = matrix[i][j]

    return padded_matrix # 제로 패딩이 적용된 행렬 반환

**이미지에 필터를 적용하는 함수**

In [422]:
def convolution(image, kernel, m, n, ksize): # size : image size, ksize : kernel size
  padded_image = zero_padding(image, ksize,m, n) # 제로 패딩 적용

  result = [[0]*n for _ in range(m)]# 컨볼루션 연산 결과를 저장할 배열 초기화

  for i in range(m): # 이미지를 순회하면서 컨볼루션 연산 수행
    for j in range(n):
      temp = 0
      for x in range(ksize):
        for y in range(ksize):
          temp += padded_image[i+x][j+y]*kernel[x][y]
      result[i][j] = temp # 컨볼루션 연산 수행 결과를 결과 행렬에 저장

  return result # 컨볼루션이 적용된 결과 이미지 반환

최종 엣지 결정 함수

In [423]:
def threshold(image, thres, m, n):
  result = [[0]*n for _ in range(m)]
  for x in range(m): # 이미지를 순회하면서 컨볼루션 연산 수행
    for y in range(n):
      # 이미지에서 thres 초과하는 픽셀은 255로, 이하는 0으로 변환
      if (image[x][y] > thres) :
        result[x][y] = 255
      else :
        result[x][y] = 0
  return result

## 엣지 검출

In [437]:
for i in range(10):
  # 이미지 로드
  image_path = f'./{i}.png'
  image = np.array(Image.open(image_path).convert('L'))

  sigma = 2.0
  m, n = image.shape # image의 세로, 가로 크기
  gsize = 7 # 가우시안 필터는 gsize*gsize 크기.

  # LoG 필터 적용
  gaussian = gaussian_kernel(sigma, gsize) # 가우시안 필터 생성
  laplacian = laplacian_kernel() # 라플라시안 커널 생성
  Log_filter = convolution(gaussian, laplacian, gsize, gsize, 3) # LoG 필터(Laplacian Of Gaussian) 생성
  Log_converted = convolution(image, Log_filter, m, n, gsize) #LoG 필터 적용

  Log_result = threshold(Log_converted, 0, m, n) # 임계값(0)을 넘은 픽셀에 대해서만 edge로 판정하여 남김

  # Log 필터 적용 결과 이미지 저장하기
  img = Image.fromarray(np.uint8(Log_result), 'L')
  img.save(f'./Log_converted_{i}.png')

  print(f'{i} finished')

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished


## new idea

In [433]:
def my_kernel():
    kernel = [[1,1,1,1,1], [1,-1,-1,-1,1], [1,-1,-8,-1,1], [1,-1,-1,-1,1], [1,1,1,1,1]]
    return kernel

In [436]:
for i in range(10):
  # 이미지 로드
  image_path = f'./{i}.png'
  image = np.array(Image.open(image_path).convert('L'))

  sigma = 2.0
  m, n = image.shape # image의 세로, 가로 크기
  gsize = 7 # 가우시안 필터는 gsize*gsize 크기.

  # 만든 필터 적용
  mykernel = my_kernel() # 커스텀 커널 생성
  gaussian = gaussian_kernel(sigma, gsize) # 가우시안 필터 생성
  filter = convolution(gaussian, mykernel, gsize, gsize, 5) # gaussian과 mykernel 컨볼루션
  my_converted = convolution(image, filter, m, n, gsize) # 필터 적용

  result = threshold(my_converted, 5, m, n)

  img = Image.fromarray(np.uint8(result), 'L')
  img.save(f'./My_converted_{i}.png')

  print(f'{i} finished')

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
